<a href="https://colab.research.google.com/github/woranov/amlta-project/blob/main/notebooks/glossary_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
try:
    import amlta
except ImportError:
    %pip install --no-dependencies git+https://github.com/woranov/amlta-project.git

%pip install bm25s

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00


In [85]:
import re
import warnings
from pathlib import Path

try:
    from google.colab import drive  # pyright: ignore[reportMissingImports]

    IN_COLAB = True
except ImportError:
    IN_COLAB = False


import bm25s
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

from amlta.config import config
from amlta.probas import extract, processes, glossary

In [3]:
if IN_COLAB:
    mount_point = Path("/content/drive")
    drive_path = mount_point / "MyDrive"

    # edit
    data_dir = drive_path / "uni" / "ws2425" / "amlta" / "project" / "data"

    config.update(data_dir=data_dir)

    if not mount_point.exists():
        drive.mount(str(mount_point))

Mounted at /content/drive


In [7]:
glossary_data = glossary.read_glossary()

In [10]:
def format_glossary_entry(term, definition):
    definition = re.sub(r"\s+", " ", definition)
    return f"# {term}\n{definition}"

In [15]:
glossary_entries = [
    format_glossary_entry(term, definition) for term, definition in glossary_data.items()
]

In [ ]:
model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-base-de", # switch to en/zh for English or Chinese
    trust_remote_code=True
)

# control your input sequence length up to 8192
model.max_seq_length = 1024

In [17]:
embeddings = model.encode(glossary_entries)

In [62]:
sparse_retriever = bm25s.BM25()
sparse_retriever.index(bm25s.tokenize(glossary_entries))

Split strings:   0%|          | 0/195 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/195 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/195 [00:00<?, ?it/s]

In [150]:
def query_glossary(query: str, k=5, alpha: float = 0.2) -> str:
    query_embedding_dense = model.encode(query)
    similarities_dense = cos_sim(
        query_embedding_dense.reshape(1, -1),
        embeddings,
    )
    similarities_dense = (similarities_dense + 1) / 2

    indices, similarities_sparse = sparse_retriever.retrieve(
        bm25s.tokenize(query), sorted=False, k=len(glossary_entries)
    )

    indices = indices[0]
    similarities_sparse = similarities_sparse[0]

    ordered_similarities_sparse = np.zeros_like(similarities_sparse)
    ordered_similarities_sparse[indices] = similarities_sparse
    if ordered_similarities_sparse.sum() > 0:
        ordered_similarities_sparse /= ordered_similarities_sparse.sum()

    similarities = (
        (1 - alpha) * similarities_dense + alpha * ordered_similarities_sparse
    ).numpy().flatten()

    return np.array(glossary_entries)[np.argsort(similarities)[::-1][:k]]

In [151]:
print("\n---\n".join((query_glossary("co2", alpha=0.5))))

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

# CO2
(Kohlendioxid) ist das quantitativ wichtigste Treibhausgas (THG), das auch durch menschliche Aktivitäten (insbesondere Verbrennung fossiler Energieträger) freigesetzt wird.
---
# CH4
ist die chemische Formel für Methan (Hauptbestandteil von Erdgas, Biogas usw.). Seine Wirkung als Treibhausgas wird in den CO2-Äquivalenten einbezogen.
---
# CO2-Äquivalente
sind das Ergebnis der Aggregation von Treibhausgasen (THG) nach ihrem Treibhauspotenzial (THP) und stellen einen Indikator für das Umweltproblemfeld "Klima" dar.
---
# Aggregierte Werte (Luftemissionen)
CO2-Äquivalente sind das Ergebnis der Aggregation von Treibhausgasen (THG) nach ihrem Treibhauspotenzial (THP) und stellen einen Indikator für das Umweltproblemfeld "Klima" dar. SO2-Äquivalent ist der quantitative Ausdruck des Versauerungspotenzials, bezogen auf das "Leit"-Gas SO2. In die SO2-Äquivalente gehen neben SO2 auch die Luftschadstoffe NOx, HCl, HF, NH3 und H2S ein.
---
# Verbrennung
Verbrennung ist die Umwandlung eines B